In [4]:
import pandas as pd
import re
import os


In [5]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'outputs')

In [6]:
# 데이터 로드
data_path = os.path.join(DATA_DIR, "train.csv")
df = pd.read_csv(data_path)

# 특수 문자, 숫자, 한글 비율을 계산하는 함수들
def special_char_ratio(text):
    return len(re.findall(r'[^\w\s]', text)) / len(text) if len(text) > 0 else 0

def number_ratio(text):
    return len(re.findall(r'\d', text)) / len(text) if len(text) > 0 else 0

def hangul_ratio(text):
    return len(re.findall(r'[가-힣]', text)) / len(text) if len(text) > 0 else 0

# 각 비율 계산
df['special_char_ratio'] = df['text'].apply(special_char_ratio)
df['number_ratio'] = df['text'].apply(number_ratio)
df['hangul_ratio'] = df['text'].apply(hangul_ratio)


In [7]:
df.head()

,ID,text,target,special_char_ratio,number_ratio,hangul_ratio
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,0.093750,0.093750,0.375000
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,0.185185,0.111111,0.259259
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,0.136364,0.090909,0.545455
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,0.034483,0.103448,0.655172
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,0.071429,0.035714,0.464286


In [15]:

# 노이즈 텍스트 식별 함수
def is_noisy(row, special_threshold=0.16, number_threshold=0.2, hangul_threshold=0.55):
    return (row['special_char_ratio'] > special_threshold or 
            row['number_ratio'] > number_threshold or 
            row['hangul_ratio'] < hangul_threshold)

# 노이즈 텍스트 식별
df['is_noisy'] = df.apply(is_noisy, axis=1)

# 노이즈 텍스트만 포함하는 새로운 데이터프레임 생성
noisy_df = df[df['is_noisy']]

# 결과 출력
print(f"Total texts: {len(df)}")
print(f"Noisy texts detected: {len(noisy_df)}")
print("\nSample of noisy texts:")
print(noisy_df[['ID', 'text', 'target']])

Total texts: 2800
Noisy texts detected: 1444

Sample of noisy texts:
                       ID                               text  target
0     ynat-v1_train_00000   정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보       4
1     ynat-v1_train_00001        K찰.국DLwo 로L3한N% 회장 2 T0&}송=       3
2     ynat-v1_train_00002             m 김정) 자주통일 새,?r열1나가야1보       2
4     ynat-v1_train_00004       pI美대선I앞두고 R2fr단 발] $비해 감시 강화       6
10    ynat-v1_train_00010        oi 매력 R모h츠a열#w3약 >l·주가 고Q/진       5
...                   ...                                ...     ...
2788  ynat-v1_train_02788                 남원소식 춘>X학%단+장Rn 모집       0
2789  ynat-v1_train_02789        이총리,세4. H직 완fl지 못해E할 일 꽤;남아       2
2790  ynat-v1_train_02790  프로농구 개막 ③ 헤인즈의 1만점·양동근의 900스틸…...       3
2792  ynat-v1_train_02792        높`X#E율…}BO Q"[/선수 몸값 상승 CAO       1
2797  ynat-v1_train_02797   텔레그램+한D 등h亞서 2시간H다운…C버T정gf39종!2보       4

[1444 rows x 3 columns]


In [26]:
"""
Target을 기준으로 각각의 data 모으기
"""
df_list = [noisy_df[noisy_df['target'] == i][['ID', 'text', 'target']] for i in range(7)]

# 결과 출력
for dataframe in df_list:
    print(dataframe.head())
    print(f"Shape: {dataframe.shape}")


                      ID                                text  target
15   ynat-v1_train_00015      게시판 KISA 박민정 책임연구원 APTLD 이사 선출       0
66   ynat-v1_train_00066  UEFA 챔스리그 4강 마드리드 더비 성사…레알 vs 아...       0
79   ynat-v1_train_00079        본t카 애스턴마틴 Sp라 국내 4P…500대g 생산       0
87   ynat-v1_train_00087         수능 6?'{="원)%&G울 문O서 h개한 \꽃Z       0
113  ynat-v1_train_00113            신간 Z끗 g이`V자cJ법>{코뮤니3트?33       0
Shape: (204, 3)
                     ID                                text  target
20  ynat-v1_train_00020  추신수 타율 0.265로 시즌 마감…최지만은 19홈런·6...       1
21  ynat-v1_train_00021        KIA I수단·팬nI께하는:호kQ4M족 한마5 S최       1
28  ynat-v1_train_00028         GD n{n.강d}감독 2로축구연맹WS7(0/회부       1
29  ynat-v1_train_00029            p-축구M인천 ,qN 35c운드 r&Q{영`       1
32  ynat-v1_train_00032         박항C 매직c베트남i축구_표팀K.?^# 쏟;*:d       1
Shape: (214, 3)
                     ID                                text  target
2   ynat-v1_train_00002              m 김정) 자주통일 새,?r열1나가야1보       2
14  ynat-v

In [28]:
# 텍스트 정제 함수
def clean_text(text):
    # 특수 문자 제거 (한글, 영문, 숫자만 남김)
    text = re.sub(r'[^\w\s가-힣]', '', text)
    # 불필요한 공백 제거
    text = re.sub(r'\s+', ' ', text).strip()
    return text



# 전역 변수로 각 데이터프레임 생성
for idx, dataframe in enumerate(df_list):
    dataframe['text'] = dataframe['text'].apply(clean_text)
    dataframe.to_csv(f'./data/noised_label/noised_label_{idx}.csv', index=False)
    print(dataframe.head())
    print(f"Shape: {dataframe.shape}")


                      ID                            text  target
15   ynat-v1_train_00015  게시판 KISA 박민정 책임연구원 APTLD 이사 선출       0
66   ynat-v1_train_00066  UEFA 챔스리그 4강 마드리드 더비 성사레알 vs 아       0
79   ynat-v1_train_00079     본t카 애스턴마틴 Sp라 국내 4P500대g 생산       0
87   ynat-v1_train_00087              수능 6원G울 문O서 h개한 꽃Z       0
113  ynat-v1_train_00113            신간 Z끗 g이V자cJ법코뮤니3트33       0
Shape: (204, 3)
                     ID                          text  target
20  ynat-v1_train_00020  추신수 타율 0265로 시즌 마감최지만은 19홈런6       1
21  ynat-v1_train_00021    KIA I수단팬nI께하는호kQ4M족 한마5 S최       1
28  ynat-v1_train_00028        GD nn강d감독 2로축구연맹WS70회부       1
29  ynat-v1_train_00029           p축구M인천 qN 35c운드 rQ영       1
32  ynat-v1_train_00032          박항C 매직c베트남i축구_표팀K 쏟d       1
Shape: (214, 3)
                     ID                            text  target
2   ynat-v1_train_00002             m 김정 자주통일 새r열1나가야1보       2
14  ynat-v1_train_00014       문인 당2 4nS 민관2동7사위 X보 철거tt       2
42  ynat-v1_tr